# Reducing pointing data

During the SUMMIT-4829 run, we obtained data at lower elevation to improve the existing pointing model. 

The data was taken in a slightly different way. Instead of centering the start in the CCD and storing the data for the pointing model we simply ran the wavefront analysis to collimate the optics, registered the position in the pointing and took an acquisition image, so we could later measure the offset and apply that to the registered position.

This notebooks is intended to gather the information about the data taken during the run. Basically we need to all the registered positions in associated acquisition images.

The next step will be to measure the position of the brightest start in the field, compute the offset with respect to the center of the field and add that to the data generated by the pointing. This will be done on a separate notebook.

In [ ]:
import numpy as np
import pandas as pd

from astropy.time import Time
from astropy import units as u
from datetime import timedelta, datetime

import lsst_efd_client

import matplotlib.pyplot as plt

from pandas.plotting import register_matplotlib_converters

from lsst.geom import PointD

from lsst.pipe.tasks.quickFrameMeasurement import QuickFrameMeasurementTask

from lsst.ts.observing.utilities.auxtel.latiss.getters import get_image
from lsst.ts.observing.utilities.auxtel.latiss.utils import parse_obs_id
from lsst.ts.observatory.control.constants.latiss_constants import boresight #, pixel_scale
pixel_scale = 0.09569



In [ ]:
%matplotlib inline

In [ ]:
client = lsst_efd_client.EfdClient('ldf_stable_efd')

## Define timestamp when the data was taken

The next cell defines the dates when the data was taken, it is used by tthe following query to determine when to look for pointing component data registration

In [ ]:
start = Time('2021-02-18T00:00:00')
end = Time('2021-02-20T00:00:00')

Look from when pointAddData command was sent to the pointing. These will mark the times when we registered the positions.

In [ ]:
timestamp = f"time >= '{start}+00:00' AND time <= '{end}+00:00'"
query = f'SELECT "private_host" FROM "efd"."autogen"."lsst.sal.ATPtg.command_pointAddData" WHERE {timestamp}'
query_ack = f'SELECT "cmdtype", "ack" FROM "efd"."autogen"."lsst.sal.ATPtg.ackcmd" WHERE {timestamp} AND cmdtype = 24 AND ack != 300'

In [ ]:
print(query)
print(query_ack)

In [ ]:
point_add_data = await client.influx_client.query(query)

In [ ]:
point_add_data_ack = await client.influx_client.query(query_ack)

In [ ]:
len(point_add_data_ack), len(point_add_data)

In [ ]:
command_ok = []
for i in range(len(point_add_data)):
    if point_add_data_ack.ack[i] == 303:
        print(f"Command OK: {point_add_data.private_host[i]}")
        command_ok.append(i)
    else:
        print(f"Command FAILED: {point_add_data.private_host[i]}")

In [ ]:
point_add_data_ok = point_add_data.loc[point_add_data.index[command_ok]]

## Finding acquisition images

Now we have the timestamps for when the telescope position was registered, we need to find the acquisition images.

The images where taken before registering the position so we need to look ~40s before the command was sent.

In [ ]:
acq_obsid = []
elevation = []
rotator_1 = []
rotator_2 = []
for time_reg in point_add_data_ok.index:
    obsid = await client.select_time_series('lsst.sal.ATArchiver.logevent_imageInOODS', ["obsid"], 
                                     Time(time_reg).tai - timedelta(seconds=40), Time(time_reg).tai)
    mount_Nasmyth_Encoders = await client.select_packed_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders",
                                                                    ["nasmyth1CalculatedAngle", 
                                                                     "nasmyth2CalculatedAngle"],
                                                                    Time(time_reg).tai - timedelta(seconds=40), 
                                                                    Time(time_reg).tai)
    rotator_1.append(np.mean(mount_Nasmyth_Encoders["nasmyth1CalculatedAngle"]))
    rotator_2.append(np.mean(mount_Nasmyth_Encoders["nasmyth2CalculatedAngle"]))
    elevationCalculatedAngle = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders",
                                                                      ["elevationCalculatedAngle"],
                                                                      Time(time_reg).tai - timedelta(seconds=40), 
                                                                      Time(time_reg).tai
                                                                     )
    elevation.append(np.mean(elevationCalculatedAngle["elevationCalculatedAngle"]))
    acq_obsid.append(obsid.obsid[-1])

## Measuring star position on each image

In [ ]:
def rotation_matrix(angle):
    """Rotation matrix.
    """
    return np.array(
        [
            [np.cos(np.radians(angle)), -np.sin(np.radians(angle)), 0.0],
            [np.sin(np.radians(angle)), np.cos(np.radians(angle)), 0.0],
            [0.0, 0.0, 1.0],
        ]
    )

In [ ]:
qm_config = QuickFrameMeasurementTask.ConfigClass()

In [ ]:
qm = QuickFrameMeasurementTask(config=qm_config)

In [ ]:
brightest_source_centroid = []

for obs_id in acq_obsid:
    day_obs, seq_num = parse_obs_id(obs_id)[-2:]
    exp = await get_image(
            dict(dayObs=day_obs, seqNum=seq_num),
            datapath="/project/shared/auxTel/",
            timeout=10,
            runBestEffortIsr=True,
        )
    result = qm.run(exp)
    brightest_source_centroid.append(result)

In [ ]:
angle = np.array(elevation) - np.array(rotator_2) + 90.0

## Compute azel offset

Each entry in `brightest_source_centroid` contains the centroid of the brightest source in the field in pixel coordinates.

We need to compute the distance to the center of the field and then convert that from xy into azel. It is this azel offset that we need to apply to the pointing table.


In [ ]:
azel_correction = np.zeros((2,len(brightest_source_centroid)))
for i, source_xy in enumerate(brightest_source_centroid):
    xy_distance = (PointD(*source_xy.brightestObjCentroid)-boresight)*pixel_scale*u.arcsec
    azel_offset = np.matmul(xy_distance.to(u.deg), rotation_matrix(-angle[i])[:2,:2])
    azel_correction[0][i] = azel_offset[0].value
    azel_correction[1][i] = azel_offset[1].value

## Apply correction to pointing data

Now that the offsets are computed in az/el, we need to read the data from the pointing table and apply the offset to the appropriate columns. 

In [ ]:
pointing_file = "data/20210219/AT_point_file_202102.dat"

Skip the first `skiprows=4` lines of the file, which contain header information.

Read `max_rows=azel_correction.shape[1]`. This avoids the issue of the last line in the pointing file being `END`.

In [ ]:
pointing_file_data = np.loadtxt(pointing_file, skiprows=4, max_rows=azel_correction.shape[1], unpack=True)

The data that needs correcting are the 3rd and 4th columns, which contains the correct az and el of the telescope. 

In [ ]:
pointing_file_data[2] += azel_correction[0]
pointing_file_data[3] += azel_correction[1]

We need to add the header and tail of the pointing file.

In [ ]:
header = """LSST Auxiliary Telescope, 2021 Feb 19 UTC 1 41 1
: ALTAZ
: ROTNL
-30 14 40.3
"""
tail = "END"
with open("data/20210219/AT_point_file_202102_rev001.dat", "w") as fp:
    fp.write(header)
    np.savetxt(fp, pointing_file_data.T, fmt="%011.7f")
    fp.write(tail)

## End

The file is now ready to be analysed with tpoint to produce a new pointing model.